# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
food_db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(food_db.list_collection_names())

['restaurants', 'establishments']


In [6]:
# review a document in the establishments collection
pprint(food_db.establishments.find_one())

{'_id': ObjectId('67807391323af9eebd724142'),
 'geocode': {'latitude': None, 'longitude': None},
 'location': 'London',
 'name': 'Sample Restaurant',
 'rating': 5.0}


In [7]:
# assign the collection to a variable
establishments = food_db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('6785a9ae602f1800012f02f0'), acknowledged=True)

In [10]:
# Check that the new restaurant was inserted
establishments.find_one({'BusinessName':'Penang Flavours'})

{'_id': ObjectId('6780795d323af9eebd724161'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 0.08384, 'latitude': 51.490142},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = ['BusinessTypeID', 'BusinessType']

establishments.find_one(query, fields)

{'_id': ObjectId('6780795d323af9eebd724161'),
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    new_restaurant, 
    {'$set': 
        {'BusinessTypeID': 1}
    }
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [13]:
# Confirm that the new restaurant was updated
establishments.find_one({'BusinessName':'Penang Flavours'})

{'_id': ObjectId('6780795d323af9eebd724161'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 0.08384, 'latitude': 51.490142},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
document = establishments.find_one()
print(document)


{'_id': ObjectId('67807391323af9eebd724142'), 'name': 'Sample Restaurant', 'location': 'London', 'rating': 5.0, 'geocode': {'latitude': None, 'longitude': None}}


In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({'LocalAuthorityName': 'Dover'})

0

In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': 'Dover'})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [17]:
# Check if any remaining documents include Dover
pprint(establishments.count_documents({'LocalAuthorityName': 'Dover'}))

0


In [18]:
# Check that other documents remain with 'find_one'
establishments.find_one({})

{'_id': ObjectId('67807391323af9eebd724142'),
 'name': 'Sample Restaurant',
 'location': 'London',
 'rating': 5.0,
 'geocode': {'latitude': None, 'longitude': None}}

5. Some of the number values are stored as strings, when they should be stored as numbers.

In [19]:
# Change the data type from String to Decimal for longitude
# Change the data type from String to Decimal for latitude
establishments.update_many({}, [{'$set': {'geocode.longitude': {'$toDouble': '$geocode.longitude'}, 
                                         'geocode.latitude': {'$toDouble': '$geocode.latitude'}
                                         }
                                }
                               ]
                          )

UpdateResult({'n': 17, 'nModified': 2, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [20]:
# Check that the coordinates are now numbers
[doc for doc in establishments.find({}, ['geocode.longitude', 'geocode.latitude'])]

[{'_id': ObjectId('67807391323af9eebd724142'),
  'geocode': {'latitude': None, 'longitude': None}},
 {'_id': ObjectId('678074c9323af9eebd724145'),
  'geocode': {'latitude': None, 'longitude': None}},
 {'_id': ObjectId('6780750e323af9eebd724147'),
  'geocode': {'latitude': None, 'longitude': None}},
 {'_id': ObjectId('6780751d323af9eebd724149'),
  'geocode': {'latitude': None, 'longitude': None}},
 {'_id': ObjectId('67807621323af9eebd72414e'),
  'geocode': {'latitude': None, 'longitude': None}},
 {'_id': ObjectId('6780765b323af9eebd724157'),
  'geocode': {'latitude': None, 'longitude': None}},
 {'_id': ObjectId('6780766c323af9eebd72415a'),
  'geocode': {'latitude': None, 'longitude': None}},
 {'_id': ObjectId('678078ca323af9eebd72415f'),
  'geocode': {'latitude': None, 'longitude': None}},
 {'_id': ObjectId('6780795d323af9eebd724161'),
  'geocode': {'longitude': 0.08384, 'latitude': 51.490142}},
 {'_id': ObjectId('678079a8323af9eebd724162'),
  'geocode': {'longitude': 0.08384, 'latitude

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

Use `update_many` to convert `RatingValue` to integer numbers.